In [1]:
import datasets
lang_pair='zh-en'
data = datasets.load_dataset("./data/datasets/datasets/wmt19",name=lang_pair,cache_dir="./data")

Found cached dataset wmt19 (/DATA/jupyter/personal/translation/./data/wmt19/zh-en/1.0.0/29e210fae5690e843cae5dc43b53db36c4e02f927db50cd5235a22ab42dde90a)


  0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
len(alt_dataset['train'])

18094

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer,tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import json
import torch
import jieba
train = data['train']
validation = data['validation']
for i,item in enumerate(train):
    print(item['translation'])
    if i == 3:
        break
eng_data = []
zh_data = []
for i,item in enumerate(train):
    english = item['translation']['en']
    english = " ".join(nltk.word_tokenize(english.lower()))
    eng_data.append(english)
    chinese = item['translation']['zh']
    chinese = " ".join(jieba.cut(str(chinese.replace(" ","")), cut_all=False, HMM=True))
    zh_data.append(chinese)
print("finished")


KeyboardInterrupt



In [3]:
val_eng_data=[]
val_zh_data=[]
for i,item in enumerate(validation):
    english = item['translation']['en']
    english = " ".join(nltk.word_tokenize(english.lower()))
    val_eng_data.append(english)
    val_chinese = item['translation']['zh']
    val_chinese = " ".join(jieba.cut(str(val_chinese.replace(" ","")), cut_all=False, HMM=True))
    val_zh_data.append(val_chinese)
print("finished")

finished


In [9]:
eng_data_js = []
zh_data_js = []
import json
with open("./data/en-zh/translation2019zh_train.json") as f:
    lines = f.read().splitlines()
for data in lines:
    json_data = json.loads(data)
    english = json_data['english']
    english = " ".join(nltk.word_tokenize(english.lower()))
    eng_data_js.append(english)
    chinese = json_data['chinese']
    chinese = " ".join(jieba.cut(str(chinese.replace(" ","")), cut_all=False, HMM=True))
    zh_data_js.append(chinese)
print("finished")

In [15]:
zh_word = []
for i in zh_data:
    field = i.split(" ")
    
    tmp = []
    for j in field:
        if text_utils(j,'zh'):
            if len(j) != 1:
                x = [k for k in j]
                tmp.extend(x)
            else:
                tmp.append(j)
        else:
            tmp.append(j)
    
    zh_word.append(" ".join(tmp))
zh_word[:3]

['1929 年 还 是 1989 年 ?',
 '巴 黎 - 随 着 经 济 危 机 不 断 加 深 和 蔓 延 ， 整 个 世 界 一 直 在 寻 找 历 史 上 的 类 似 事 件 希 望 有 助 于 我 们 了 解 目 前 正 在 发 生 的 情 况 。',
 '一 开 始 ， 很 多 人 把 这 次 危 机 比 作 1982 年 或 1973 年 所 发 生 的 情 况 ， 这 样 得 类 比 是 令 人 宽 心 的 ， 因 为 这 两 段 时 期 意 味 着 典 型 的 周 期 性 衰 退 。']

['1929 年 还是 1989 年 ?',
 '巴黎 - 随着 经济危机 不断 加深 和 蔓延 ， 整个 世界 一直 在 寻找 历史 上 的 类似 事件 希望 有助于 我们 了解 目前 正在 发生 的 情况 。',
 '一 开始 ， 很多 人 把 这次 危机 比作 1982 年 或 1973 年 所 发生 的 情况 ， 这样 得 类比 是 令人 宽心 的 ， 因为 这 两段 时期 意味着 典型 的 周期性 衰退 。']

In [16]:
print(len(eng_data))
with open("./data/en-zh/wmt19.zh","a+") as f:
    for i in zh_word:
        f.write(i+"\n")

25984574


In [ ]:
def format_translation_data(translations, tokenizer, seq_len=128):
  
    input_text = translations['en']
    target_text = translations['zh']

    if input_text is None or target_text is None:
        return None

    input_token_ids = encode_input_str(
        input_text, target_lang, tokenizer, seq_len, lang_token_map)
  
    target_token_ids = encode_target_str(
        target_text, tokenizer, seq_len, lang_token_map)

    return input_token_ids, target_token_ids

def transform_batch(batch, lang_token_map, tokenizer):
    
    inputs = []
    targets = []
    for translation_set in batch['translation']:
        formatted_data = format_translation_data(
             translation_set, tokenizer, max_seq_len)
    
    if formatted_data is None:
        continue
    
    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))
    
    batch_input_ids = torch.cat(inputs).cuda()
    batch_target_ids = torch.cat(targets).cuda()

    return batch_input_ids, batch_target_ids

def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=32):
    dataset = dataset.shuffle()
    for i in range(0, len(dataset), batch_size):
        raw_batch = dataset[i:i+batch_size]
        yield transform_batch(raw_batch, lang_token_map, tokenizer)

In [19]:
import jieba
def load_txt(file_path):
    data = []
    with open(file_path) as f:
        line = f.readline()
        while line:
            line = line.replace("\n","").replace(" ","")
            data.append(line)
            line = f.readline()
    return data
lines = load_txt("./data/tatoeba/train.zh")

lines = [l.replace(" ","").replace("-","").replace("","").replace("и","") for l in lines]

data = [" ".join(jieba.cut(str(s), cut_all=False, HMM=True)) for s in lines]
with open("./data/tatoeba/dev.split.zh","a+") as f:
    for i in data:
        f.write(i+"\n")

In [5]:
import jieba
from sudachipy import tokenizer
from sudachipy import dictionary

def handel_data(input_file,out_file,lang='zh'):
    count = 0
    size = 4
    
    with open(input_file) as f:
        tmp = list()
        line = f.readline()
        while line:
            count += 1
            line = line.replace(" ","").replace("-","")
            if lang == 'zh':
                data = " ".join(jieba.cut(str(line), cut_all=False, HMM=True))
            elif lang == 'ja':
                data = " ".join([m.surface() for m in tokenizer_obj.tokenize(line, mode)])
            tmp.append(data)
            if len(tmp) == size:
                # print("write",len(tmp))
                with open(out_file,"a+") as fp:
                    fp.writelines(tmp)
                tmp.clear()
            line = f.readline()
        
        
        if len(tmp) > 0:
            print("end... ")
            with open(out_file,"a+") as fp:
                fp.writelines(tmp)
        
    
    return count

number_zh = handel_data("./data/ja-zh/test.dedup.zh","./data/finall/test.zh")
number_ja = handel_data("./data/ja-zh/test.dedup.ja","./data/finall/test.ja",'ja')
print("中文句子数量：{},日文句子数量:{}".format(str(number_zh),str(number_ja)))

end... 
end... 
中文句子数量：7194,日文句子数量:7194


In [15]:
def load_txt(file_path):
    with open(file_path) as f:
        lines = f.read().splitlines()
    return lines

def write_txt(data:list,file_name):
    with open(file_name,"a+") as f:
        for i in data:
            f.write(i.replace("\n","")+"\n")
        
from opencc import OpenCC

zho = load_txt("./data/tatoeba/dev.zh")
jianti = [OpenCC("t2s").convert(s) for s in zho]
write_txt(jianti,"./data/tatoeba/dev.jianti.zh")
print("done")

done


In [17]:
from sudachipy import tokenizer
from sudachipy import dictionary

tokenizer_obj = dictionary.Dictionary().create()
with open("./data/tatoeba/test.ja")  as f: 
    lines = f.read().splitlines()

lines = [l.replace(" ","").replace("-","") for l in lines]
mode = tokenizer.Tokenizer.SplitMode.B

data = [" ".join([m.surface() for m in tokenizer_obj.tokenize(i, mode)]) for i in lines]
with open("./data/tatoeba/test.split.ja","a+") as f:
    for i in data:
        f.write(i+"\n")

In [1]:
# import sentencepiece as spm
# op = "--input=./data/finall/test.zh,./data/finall/test.ja,./data/finall/train.ja,./data/finall/train.zh --model_prefix=./sentpiece.bpe --vocab_size=15680 --character_coverage=1.0 --model_type=bpe"
# spm.SentencePieceTrainer.Train(op)

In [100]:
from pysubs2 import SSAFile, SSAEvent, make_time
import os
all = []
files=[]
for f in os.listdir("./data/zimu"):
    remove_fil="1850年代出版的一部有历史背景的小说"
    try:
        data={}
        if f.endswith(".ass") or f.endswith(".srt") or f.endswith(".ssa"):
            subs = SSAFile.load('./data/zimu/{}'.format(f))

            for line in subs:
                time = str(line.start)
                text = line.plaintext
                tmp = {"text1":"","text2":""}
                
                if time not in data:
                    tmp["text1"] = text
                    data[time] = tmp
                else:
                    data[time]["text2"] = text
              #print(line.plaintext)
              #print(line.start)
            files.append(f)
            all.append(data)
    
    except:
        print("error",f,text)
        continue

print(len(files))

1


In [93]:
def load_diff_file():
    zh=[]
    ja=[]
    files=[]
    for f in os.listdir("./data/zimu"):
        try:
            data_zh = []
            data_ja = []
            if f.endswith(".ass") or f.endswith(".srt") or f.endswith(".ssa"):
                post_f = f[3:]
                
                chs = SSAFile.load('./data/zimu/Chs{}'.format(post_f))
                jpn = SSAFile.load('./data/zimu/Jpn{}'.format(post_f))
              
                for ch in chs:
                    txt = ch.plaintext
                    data_zh.append(txt.replace("\n"," "))
                for jn in jpn:
                    txt = jn.plaintext
                    data_ja.append(txt.replace("\n"," "))
                zh.append(data_zh)
                ja.append(data_ja)
               
                files.append(f)
                break
        except:
            print("error",f)
            continue
    print(len(files))
    print(len(zh))
    print(len(ja))
    return zh,ja


In [94]:
chs,jpn = load_diff_file()
print(chs[0][:5])
print(jpn[0][:5])

1
1
1
['要反抗吗？', '和我一起', '要不要起身反抗？', '（关东地区沉没的可能性）', '什么？']
['（天海(あまみ)） 戦ってみないか？', '一緒に', '（天海）戦ってみないか', '（常盤(ときわ)）なんで…', '（東山(ひがしやま)）誰が…']


In [71]:
# ass_zh = []
# ass_ja = []
# for data in file_data:
#     for k,la in data.items():
#         if len(la['zh']) > 0 and len(la['ja']):
#             ass_zh.append(la['zh'])
#             ass_ja.append(la['ja'])
# print(len(ass_zh))
# print(len(ass_ja))
# print(ass_zh[:5])
# print(ass_ja[:5])
# print(ass_zh[-10:])
# print(ass_ja[-10:])

In [21]:
import re
jap = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\uAC00-\uD7A3]')
chinese=re.compile(u'[\u4e00-\u9fa5]+')

def text_utils(text,lang='ja'):
    if lang.lower()=='ja':
        jap = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\uAC00-\uD7A3]') #判断韩文跟日文
        return bool(jap.search(text))
    elif lang.lower() == 'en':
        return bool(re.search('[a-z]',text))
    elif lang.lower() == 'zh':
        chinese=re.compile(u'[\u4e00-\u9fa5]+')
        return bool(chinese.search(text))
        
text_utils("Now, your meaningless life","en")

True

In [114]:
# print(all[0])
# print(files[0])

zh=[]
ja=[]
import re
jap = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\uAC00-\uD7A3]')
chinese=re.compile(u'[\u4e00-\u9fa5]+')
src_lang='ja'
  

for it in all:
    for k,v in it.items():
        if text_utils(v['text1'],src_lang) and len(v["text2"]) > 0:
            txt = v['text1']
            wen=txt.replace("\u3000"," ")
            
            items = wen.split("\n")
            
            if len(items) == 2:
                if text_utils(items[0],src_lang) and not text_utils(items[0],'zh'):
                 
                    ja.append(items[0])
                    zh.append(items[1])
                else:
                    
                    ja.append(items[1])
                    zh.append(items[0])
        
            if len(items) ==1 and chinese.search(v["text2"]):
                ja.append(wen)
                zh.append(v['text2'])
                # print(v['text2'])
            
    
                
        elif text_utils(v['text1'],src_lang) and len(v["text2"]) == 0:
            txt = v['text1']
            wen=txt.replace("\u3000"," ")
            items = wen.split("\n")
            
            if len(items) == 2:
                if text_utils(items[0],src_lang): #and not text_utils(items[0],'zh')
                 
                    ja.append(items[0])
                    zh.append(items[1])
                else:
                    print(items[0])
                    ja.append(items[1])
                    zh.append(items[0])
                    
            if len(items) > 2:
                if text_utils(items[0],'zh') and not text_utils(items[0],src_lang):
                    print(items)
                    zh.append(items[0])
                    ja.append("".join(items[1:]))
            
        else:
            
            wen=v['text2'].replace("\u3000"," ")
            if text_utils(wen,src_lang):
                # print("hello")
                if len(wen) > 0:
                    ja.append(wen)
                    zh.append(v['text1'])



print(len(zh) == len(ja))
print(len(zh))
print(zh[-20:])
print(ja[-20:])
print(zh[2:20])
print(ja[2:20])

True
505
['在看见什么的一瞬间', '空气流动着', '在杂乱的日常生活中', '呼吸的空气中 飞舞着灰尘', '是因为什么都没有改变的原因吧', '空气都静止了…', '身体疲惫不堪', '对这样的自己 没有任何的感动', '孤独的时候 好想找人说说话', '今天就让我们悠闲的交谈吧', '你展现出这一天', '最平和的表情', '在这疲惫不堪的紧张感中', '展现出最灿烂的时刻', '今天就让我们悠闲的交谈吧', '你展现出这一天', '最平和的表情', '在众多的愿望中', '如果只能实现其中一个的话', '我会选择什么呢…']
['そんな何かを見た瞬間に', '空気が動く', '雑然とした日常の中で', '息を吸うたび舞うホコリのように', '何ら 変わっちゃいないせいなのさ', '空気が止まる…', 'くたびれていて', 'そんな自分が無感動で', '人恋しくて 孤独なら', '今日はゆっくり話そう', '君は この日一番の穏やかな', 'その顔を 見せるね', 'すり切れる程の 緊張感の中で', '最も輝くその時を', '今日はゆっくり話そう', '君は この日一番の穏やかな', 'その顔を 見せるね', '数多い願いの中でも', 'たったひとつ叶えられるとしたら', '何を選ぶだろう…']
['想在你的梦里和你继续走下去', '夏天悄无声息的来临', '波光粼粼的海浪 浸湿了沙滩', '将周围的一切束缚全都抛开', '现在 就决定是你了', '这样自己没有合适的另一半', '心曾死了一半', '摇曳的思绪      荡漾在心头', '想要像现在这样一直陪在你身边', '就像那湛蓝而又清澈的天空', '想在你的梦里和你继续走下去', '你的眼眸深处传递出了喜欢的讯息', '在假装看你双眸的时候 偷吻一下', '因为害怕将自己完全暴露在你的面前', '所以有时也会逃避你的温柔', '这是命运的相逢 毋庸置疑', '自己竟也开始改变', '摇曳的思绪      荡漾在心头']
['君と歩き続けたい in your dream', '夏が忍び足で 近づくよ', 'きらめく波が 砂浜 潤して', 'こだわってた周囲を すべて捨てて', '今 あなたに決めたの', 'こんな自分に合う人はもう', 'いないと半分あきらめてた', '揺れる想い体じゅう感じて

In [112]:
# all[0]


In [345]:
zh=[]
ja=[]
import re
jap = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\uAC00-\uD7A3]')

for it in all[:2]:
    for k,v in it.items():
        
        if text_utils(v['text1'],'en'):
            txt = v['text1'].split("\n")
        
        if len(txt) == 2:
            if text_utils(txt[0],'en'):
                wen=txt[0].replace("\u3000"," ")
                ja.append(wen)
                zh.append(txt[1])
            else:
                wen=txt[1].replace("\u3000"," ")
                ja.append(wen)
                zh.append(txt[0])
print(zh[:10])
print(ja[:10])
print(len(ja)==len(zh))

['--==本影片由 CMCT 團隊 榮譽出品==--', '--==本影片由 CMCT 团队 荣誉出品==--']
['更多精彩影視 請訪問 http://cmct.cc', '更多精彩影视 请访问 http://cmct.cc']
True


In [6]:
from opencc import OpenCC
zh = ['碞稰谋硂赣']
jianti = [OpenCC("t2s").convert(s) for s in zh]
print(jianti)

['碞稰谋硂赣']


In [115]:
tgt_a = []
src_a = []
jianti=zh
for i in range(len(jianti)):
    if "翻译" not in ja[i]:
        if "第5集中也曾" not in ja[i]:
            tgt_a.append(jianti[i].replace("\ufeff","").replace("♪","").replace("\t",""))
            src_a.append(ja[i].replace("\ufeff","").replace("♪","").replace("\t",""))
print(len(tgt_a)==len(src_a))
print(tgt_a[:17])
print(src_a[:17])
print(tgt_a[-10:])
print(src_a[-10:])
print(len(src_a))

True
['摇曳的思绪', '摇曳的思绪      荡漾在心头', '想在你的梦里和你继续走下去', '夏天悄无声息的来临', '波光粼粼的海浪 浸湿了沙滩', '将周围的一切束缚全都抛开', '现在 就决定是你了', '这样自己没有合适的另一半', '心曾死了一半', '摇曳的思绪      荡漾在心头', '想要像现在这样一直陪在你身边', '就像那湛蓝而又清澈的天空', '想在你的梦里和你继续走下去', '你的眼眸深处传递出了喜欢的讯息', '在假装看你双眸的时候 偷吻一下', '因为害怕将自己完全暴露在你的面前', '所以有时也会逃避你的温柔']
['揺れる想い', '揺れる想い体じゅう感じて', '君と歩き続けたい in your dream', '夏が忍び足で 近づくよ', 'きらめく波が 砂浜 潤して', 'こだわってた周囲を すべて捨てて', '今 あなたに決めたの', 'こんな自分に合う人はもう', 'いないと半分あきらめてた', '揺れる想い体じゅう感じて', 'このままずっとそばにいたい', '青く澄んだあの空のような', '君と歩き続けたい in your dream', '好きと合図送る 瞳の奥', '覗いてみる振りして キスをした', 'すべてを見せるのが 怖いから', 'やさしさから逃げてたの']
['你展现出这一天', '最平和的表情', '在这疲惫不堪的紧张感中', '展现出最灿烂的时刻', '今天就让我们悠闲的交谈吧', '你展现出这一天', '最平和的表情', '在众多的愿望中', '如果只能实现其中一个的话', '我会选择什么呢…']
['君は この日一番の穏やかな', 'その顔を 見せるね', 'すり切れる程の 緊張感の中で', '最も輝くその時を', '今日はゆっくり話そう', '君は この日一番の穏やかな', 'その顔を 見せるね', '数多い願いの中でも', 'たったひとつ叶えられるとしたら', '何を選ぶだろう…']
505


In [27]:
src_b = []
tgt_b = []
for id,en in enumerate(src_a):
    if text_utils(en,"zh"):
        its = en.split("\n")
        if len(its) == 2:
            if text_utils(its[0],src_lang):
                src_b.append(its[0])
                tgt_b.append(its[1])
            else:
                src_b.append(its[1])
                tgt_b.append(its[0])
     
    else:
        src_b.append(src_a[id])
        tgt_b.append(tgt_a[id])

print(len(src_b))
print(src_b[:30])
print(tgt_b[:30])
print(src_b[-20:])
print(tgt_b[-20:])

548
['Ryokee\nりょうっき', 'マジでやるんだな', 'どこからがいい メガネ', 'つまらんヤツだ', 'それが IBM', 'どういうことだ', 'そうだったのか', 'ちくしょう', 'テレビもくれ', 'おい こらっ', 'どうすんだよ', 'もういい ずっとそうやってろ', 'よし じゃあついてこい', 'ひょっとしてこいつ…', 'ありがとう', 'よかった…かね', 'そうなんだ', 'うん', 'におうんだよ', 'バカにしてるのか', 'ダメです', 'そ…それは…', 'どうだった', 'となると やはり…', 'はい', 'コメント それだけですか', 'ここは…', 'こんにちはー', 'マズいな', 'それは…']
['\n时间轴/特效', '真要干啊', '眼镜兄 你想先听什么', '无趣的家伙', '那就是IBM', '这是什么意思', '原来是这样', '可恶', '再给我弄台电视来', '喂 别走', '你到底想怎样', '好好好 你就一直这样吧', '好 那跟我来吧', '难道说这家伙…', '谢谢您', '以前…挺好的', '原来是这样', '嗯', '我能闻到', '你把我们当傻子吗', '这可不行', '那…那是…', '怎么样', '这么说 果然…', '是的', '您要说的只有这些吗', '这里是…', '您好', '糟了', '那是因为…']
['ケイ', 'なに', 'コレってやっぱりこういうことなんだ', 'しくったぜ', 'うん では', 'はい？', 'はッ', 'はい', 'どうした', 'あ…いや', 'いきなりこんなことになるなんて', 'そんなこと…それよりケガ…', 'クソッ', 'ついてる', 'だよな', 'おい ふざけんな', 'エンストか', 'なんだろう', 'さっきのでハッキリした', 'すごく…おしっこしたい']
['圭', '怎么回事', '果然是这么一回事', '搞砸了', '好 失陪了', '什么？', '是', '是', '怎么了', '啊…没事', '没想到会发生这种事', '没事…比起这个 你的伤…', '可恶', '跟过来了', '当然了', '喂 别开玩笑了', '熄火了吗', '那边怎么了', '刚才的事让我明白了', '好想…小便']


In [721]:
#

In [117]:
with open("./data/videozimu/xunchang.ja","a+") as f:
    for i in src_a:
        f.write(i.replace("\n","")+"\n")


In [15]:
pr_en_file="./data/en-zh/{}".format("en-zh.en")
pr_zh_file="./data/en-zh/{}".format("en-zh.zh")
with open(pr_en_file) as f:
    src_b = f.read().splitlines()
with open(pr_zh_file) as f:
    tgt_b = f.read().splitlines()
src_all_b=[i.strip().replace('"',"") for i in src_b]
tgt_all_b=[i.strip() for i in tgt_b]
src_b = src_all_b[:40000]
tgt_b = tgt_all_b[:40000]
print(len(src_b))
print(len(tgt_b))
print(src_b[:10])
print(tgt_b[:10])
ori_en = src_all_b[40000:]
ori_zh = tgt_all_b[40000:]
print(len(ori_en))
print(len(ori_zh))

40000
40000
["There's hundreds of them. Shall we go get them?", 'Yeah. This will be fun.', 'Yeah, absolutely.', 'He makes it look so easy on those ski-doos.', "But in virgin snow, you know, it's incredibly difficult,", "because the slower you go, the more you're gonna fall off.", "Oh, that's hard!", "You've gotta get speed", 'to get momentum to sit on top of that snow.', "It looks easy, but it's bloody tough."]
['得有几百只吧  我们去抓吧', '是的  会很有趣的', '肯定的', '看他骑雪地摩托感觉很轻松', '但在没人走过的雪地里  行动非常困难', '因为你开得越慢  就越容易摔下来', '这太难了', '你必须得有速度', '才能获得在雪上骑行的足够动力', '看起来简单  但十分困难']
58145
58145


In [18]:
tgt_c=[]
src_c=[]
src_tmp = ""
tgt_tmp = ""
for i,en in enumerate(src_b):
    
    en = en.replace("...","")
    if en.endswith(".") or en.endswith("?") or en.endswith("!"):
        
        tgt_tmp = tgt_tmp + " " + tgt_b[i]
        src_tmp = src_tmp + " " + en
        tgt_c.append(tgt_tmp)
        src_c.append(src_tmp)
        src_tmp = ""
        tgt_tmp = ""
    else:
        tgt_tmp = tgt_tmp + " " + tgt_b[i]
        src_tmp = src_tmp + " " + en
        
if len(tgt_tmp) > 0:
    tgt_c.append(tgt_tmp)
    src_c.append(src_tmp)
        
#     if en.startswith("of") or en.startswith("and") or en.startswith("or"):
#         tgt_tmp = tgt_tmp + " " + tgt_b[i]
#         src_tmp = src_tmp + " " + en
    
src_c=[i.strip().replace('"',"") for i in src_c]
tgt_c=[i.strip() for i in tgt_c]
print(len(tgt_c))
print(len(tgt_c)==len(src_c))
print(tgt_c[:20])
print(src_c[:20])
print(tgt_c[-20:])
print(src_c[-20:])

32495
True
['得有几百只吧  我们去抓吧', '是的  会很有趣的', '肯定的', '看他骑雪地摩托感觉很轻松', '但在没人走过的雪地里  行动非常困难 因为你开得越慢  就越容易摔下来', '这太难了', '你必须得有速度 才能获得在雪上骑行的足够动力', '看起来简单  但十分困难', '上到山顶上之后 风突然变大了', '我在挪威美丽的西海岸上', '这里峡湾遍布  寒冷刺骨 还有身材健硕的北欧海盗', '我一直很喜欢这个 伟大国家生产的海鲜', '所以我在十二月份来到了这里', '大厨告诉我这个时候是挪威海鲜 和其他美味佳肴的巅峰时期', '所以我要唤起我内心的维京人 在这寒冷的温度里生存下去', '我该开始了  因为现在这里的平均日照 只有大约五个小时', '该出发了', '我的挪威冒险 始于卑尔根外的小岛上 就在挪威的西南海岸 我和克里斯托弗·哈图夫主厨 在他最爱的觅食点碰面', '这里太漂亮了', '欢迎']
["There's hundreds of them. Shall we go get them?", 'Yeah. This will be fun.', 'Yeah, absolutely.', 'He makes it look so easy on those ski-doos.', "But in virgin snow, you know, it's incredibly difficult, because the slower you go, the more you're gonna fall off.", "Oh, that's hard!", "You've gotta get speed to get momentum to sit on top of that snow.", "It looks easy, but it's bloody tough.", 'You get to the top of the ridge on that mountain, and the wind just kicks in.', "I'm in norway on the beautiful west coast.", 'Home to fjords, freezing temperatures, and bloody

In [9]:
a = [1,2,3,4,5,6,7]
step=4
for i in range(0,len(a),step):
    x = a[i:i+step]
    print(x)

[1, 2, 3, 4]
[5, 6, 7]
